In [1]:
%matplotlib inline

import Bayesian_Particle_Tracking
from Bayesian_Particle_Tracking import model
from Bayesian_Particle_Tracking.model import displacement, log_likelihood, log_posterior, log_prior, diffusion
from Bayesian_Particle_Tracking import io
from Bayesian_Particle_Tracking.prior import JeffreysPrior, UniformPrior
from Bayesian_Particle_Tracking.printable import Printable
from Bayesian_Particle_Tracking import generate_data
from Bayesian_Particle_Tracking.generate_data import data_generation, generator
import emcee
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

/Users/alanzhou/anaconda3/lib/python3.5/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [23]:
twod = generator(1000, 1e-6, 1e-10, (0,0,0), parameter = "D", ndim = 3)
print(twod.shape[1])
twod = diffusion(twod)


4


In [24]:
log_likelihood(1e-10, twod)

9210.8319246593364

In [35]:
def max_likelihood(data, lower_bound, upper_bound, intervals):
    D = np.logspace(lower_bound, upper_bound, intervals)
    loglikelihood = np.array(list(map(lambda d: log_likelihood(d, data), D)))
    maxindex = np.argmax(loglikelihood)
    confidence = D[loglikelihood > (loglikelihood.max() - 0.5)]
    return D, D[maxindex], loglikelihood, confidence.min(), confidence.max()

In [38]:
D, Dmax, loglikelihood = max_likelihood(twod, -11, -9, 10000)
Dmax

(10000,)


9.5081890855161549e-11

In [40]:
print(D[loglikelihood > (loglikelihood.max() - 0.5)].min())
print(D[loglikelihood > (loglikelihood.max() - 0.5)].max())

9.09695171563e-11
9.94717526234e-11


In [11]:
sigma = np.ones(10)*7
sigma[5] = 10
sigma

array([  7.,   7.,   7.,   7.,   7.,  10.,   7.,   7.,   7.,   7.])

In [12]:
def new_sigma(array):
    array1 = array[1:len(array)]
    array2 = array[:len(array)-1]
    array = np.sqrt(array1**2+array2**2)
    return array

def new_sigma2(array):
    array = np.sqrt(2)*array
    return array

In [13]:
print(new_sigma(sigma))
print(new_sigma2(sigma))

[  9.89949494   9.89949494   9.89949494   9.89949494  12.20655562
  12.20655562   9.89949494   9.89949494   9.89949494]
[  9.89949494   9.89949494   9.89949494   9.89949494   9.89949494
  14.14213562   9.89949494   9.89949494   9.89949494   9.89949494]


In [14]:
%timeit sigma.max()

The slowest run took 21.79 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 1.92 µs per loop


In [15]:
%timeit np.max(sigma)

The slowest run took 15.50 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 2.28 µs per loop
